# A note on Question 21 from "40 questions on probability"

## Description and exact formal solution

Question 21 from the <a href="https://www.analyticsvidhya.com/blog/2017/04/40-questions-on-probability-for-all-aspiring-data-scientists/" target="_blank">contest</a> sounds like this: which of the following events is most likely:

<ul>
<li>(A) At least 1 six when 6 dice are rolled</li>
<li>(B) At least 2 sixes when 12 dice are rolled</li>
<li>(C) At least 3 sixes when 18 dice are rolled</li>
</ul>

The solution to this question contains contains an error. Indeed, denote $p=1/6$ the probability of getting 6 in a single roll, $f(n,p,k)$ the probability of getting $k$ sixes in $n$ rolls (it is clearly binomial), and $F(n,p,k)$ the corresponding binomial CDF, so that

$$
F(n,p,k) = \sum_{i=0}^k f(n,p,i).
$$

The probabilities of events of interest would be:

$$
P(A)=\sum_{i=1}^6 f(6,p,i)=1-F(6,p,0),
$$

and

$$
P(B)=\sum_{i=2}^{12} f(12,p,i)=1-F(12,p,1),
$$

and

$$
P(C)=\sum_{i=3}^{18} f(18,p,i)=1-F(18,p,2).
$$

Calculating the probabilities implemented below.

In [1]:
import numpy as np
from scipy.stats import binom

In [2]:
pr = 1 / 6
pA = 1 - binom.cdf(0, n=6, p=pr)
pB = 1 - binom.cdf(1, n=12, p=pr)
pC = 1 - binom.cdf(2, n=18, p=pr)
print('Probabilities p(A) = {0:6.3f}, p(B) = {1:6.3f}, p(C) = {2:6.3f}'.format(pA, pB, pC))

Probabilities p(A) =  0.665, p(B) =  0.619, p(C) =  0.597


The answer is still correct, option (A) is the most likely among those mentioned.

If we slightly change the question:

<ul>
<li>(A') Exactly 1 six when 6 dice are rolled</li>
<li>(B') Exactly 2 sixes when 12 dice are rolled</li>
<li>(C') Exactly 3 sixes when 18 dice are rolled</li>
</ul>

the solution would look like this:

$$
P(A')=f(6,p,1)={6\choose1} p^1(1-p)^5
$$

and

$$
P(B')=f(12,p,2)={12\choose2} p^2(1-p)^{10}
$$

and

$$
P(C')=f(18,p,3)={18\choose3} p^3(1-p)^{15}
$$

Actual computing is implemented below, option A' is still most likely.

In [3]:
pA1 = binom.pmf(1, 6, pr)
pB1 = binom.pmf(2, 12, pr)
pC1 = binom.pmf(3, 18, pr)
print('Probabilities p(A\') = {0:6.3f}, p(B\') = {1:6.3f}, p(C\') = {2:6.3f}'.format(pA1, pB1, pC1))

Probabilities p(A') =  0.402, p(B') =  0.296, p(C') =  0.245


Note also that the solution presented at the <a href="https://www.analyticsvidhya.com/blog/2017/04/40-questions-on-probability-for-all-aspiring-data-scientists/" target="_blank">contest</a> actually solves somewhat slightly different problem: which of the following events is most likely:

<ul>
<li>(A'') 6 dice rolled, the only six appeared on the first dice</li>
<li>(B'') 12 dice rolled, the two sixes appeared on the first and second dice</li>
<li>(C'') 18 dice rolled, the three sixes appeared on the first, second and third dice</li>
</ul>

The solution to this problem is:

$$
P(A'')=p^1(1-p)^5
$$

and

$$
P(B'')=p^2(1-p)^{10}
$$

and

$$
P(C'')=p^3(1-p)^{15}
$$

The probabilities are calculated in the code below:

In [4]:
pA2 = pr * (1 - pr) ** 5
pB2 = pr ** 2 * (1 - pr) ** 10
pC2 = pr ** 3 * (1 - pr) ** 15
print('Probabilities p(A\'\') = {0:8.5f}, p(B\'\') = {1:8.5f}, p(C\'\') = {2:8.5f}'.format(pA2, pB2, pC2))

Probabilities p(A'') =  0.06698, p(B'') =  0.00449, p(C'') =  0.00030


These probabilities coincide with those obtained in the contest solution description.

## Monte Carlo simulation

### Events A, B, C

In [5]:
# prepare n_trials=100,000 Monte Carlo simulations of rolling dice
n_trials = 100000
dice = [1,2,3,4,5,6]
xA = np.random.choice(dice, size=(n_trials,6))
xB = np.random.choice(dice, size=(n_trials,12))
xC = np.random.choice(dice, size=(n_trials,18))

# variant 1, 'at least 1, 2, 3'
indA = (xA == 6).sum(axis=1)
indB = (xB == 6).sum(axis=1)
indC = (xC == 6).sum(axis=1)
pA = (indA >= 1).mean()
pB = (indB >= 2).mean()
pC = (indC >= 3).mean()
print('Probabilities p(A) = {0:6.3f}, p(B) = {1:6.3f}, p(C) = {2:6.3f}'.format(pA, pB, pC))

# variant 2, 'exactly 1, 2, 3'
indA = (xA == 6).sum(axis=1)
indB = (xB == 6).sum(axis=1)
indC = (xC == 6).sum(axis=1)
pA1 = (indA == 1).mean()
pB1 = (indB == 2).mean()
pC1 = (indC == 3).mean()
print('Probabilities p(A\') = {0:6.3f}, p(B\') = {1:6.3f}, p(C\') = {2:6.3f}'.format(pA1, pB1, pC1))

# variant 3, 'only selected equal 6'
indA = (xA[:, 0] == 6) & (~(xA[:, 1:] == 6)).all(axis=1)
indB = (xB[:, :2] == 6).all(axis=1) & (~(xB[:, 2:] == 6)).all(axis=1)
indC = (xC[:, :3] == 6).all(axis=1) & (~(xC[:, 3:] == 6)).all(axis=1)
pA2 = indA.mean()
pB2 = indB.mean()
pC2 = indC.mean()
print('Probabilities p(A\'\') = {0:8.5f}, p(B\'\') = {1:8.5f}, p(C\'\') = {2:8.5f}'.format(pA2, pB2, pC2))

Probabilities p(A) =  0.666, p(B) =  0.618, p(C) =  0.597
Probabilities p(A') =  0.402, p(B') =  0.295, p(C') =  0.244
Probabilities p(A'') =  0.06550, p(B'') =  0.00433, p(C'') =  0.00027


We see perfect concordance between theoretic and Monte Carlo results.